### Text Selection Data Merge

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 45000  # 28 native word end index

# word all usage in twogram
word_use_num_min = 1  # word usage in selected twograms 
word_use_num_max = 2

# n gram select
twogram_select_start = 0
twogram_select_end = 10000

threegram_select_start = 0
threegram_select_end = 10000

fourgram_select_start = 0
fourgram_select_end = 10000

fivegram_select_start = 0
fivegram_select_end = 10000

sixgram_select_start = 0
sixgram_select_end = 10000

sevengram_select_start = 0
sevengram_select_end = 10000

eightgram_select_start = 0
eightgram_select_end = 10000

ninegram_select_start = 0
ninegram_select_end = 10000

tengram_select_start = 0
tengram_select_end = 10000


# prefix suffix file
prefix_suffix = False  # True, False: True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
    
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Text Selection/Data/2-Text Selection Data Merge"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [6]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [7]:
def word_in_wordgroup(source_word_list, df_target, target_column, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)]   
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [8]:
def word_in_wordgroup_simple(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)]   
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Text Data Read

In [9]:
df_word_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Word_Selected_45000.xlsx")
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
44995,hayırbu,871
44996,konferansına,871
44997,kardeşisin,871
44998,kaldırıyorsun,871


In [10]:
df_twogram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Twogram_Selected_10000.xlsx")
df_twogram_select

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
9995,çok acıktım,1317
9996,etmek yok,1317
9997,veya bir,1317
9998,evli misiniz,1316


In [11]:
df_threegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Threegram_Selected_10000.xlsx")
df_threegram_select

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
9995,para var mı,701
9996,neden benim için,701
9997,neler oluyor be,701
9998,benim babam değil,701


In [12]:
df_fourgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Fourgram_Selected_10000.xlsx")
df_fourgram_select

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
9995,bir iş görüşmem var,177
9996,için biraz daha zaman,177
9997,iyi olacak tamam mı,177
9998,buna nasıl cesaret edersin,177


In [13]:
df_fivegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Fivegram_Selected_10000.xlsx")
df_fivegram_select

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
9995,neden kıza onu sevdiğini söylemiyorsun,66
9996,ve şu resme bir bak,66
9997,beni johnnynin bir dostu yolladı,66
9998,bayan teğmen dane ne yaptılar,66


In [14]:
df_sixgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Sixgram_Selected_10000.xlsx")
df_sixgram_select

,sixgram,frequency
0,sana söylemem gereken bir şey var,3646
1,bir iki üç dört beş altı,2025
2,söylemek istediğin bir şey var mı,1794
3,daha iyi bir fikrin var mı,1792
4,bunun iyi bir fikir olduğunu sanmıyorum,1743
...,...,...
9995,kurtarmak için basit oyunumu oynaman gerekecek,29
9996,bir şey olduğunu bilen bir adamsın,29
9997,artık yaşlı olduğumdan geceleri dışarı çıkmaya,29
9998,onu haklamak istiyorsan buyur bitir işini,29


In [15]:
df_sevengram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Sevengram_Selected_10000.xlsx")
df_sevengram_select

,sevengram,frequency
0,bir iki üç dört beş altı yedi,1455
1,iki üç dört beş altı yedi sekiz,1080
2,bana söylemek istediğin bir şey var mı,752
3,yedi altı beş dört üç iki bir,596
4,sekiz yedi altı beş dört üç iki,551
...,...,...
9995,demek istediğini anlıyorum tamam parayı o aldı,23
9996,sizin gibi böyle yakışıklı genç biri için,23
9997,kıçını halledecek olan bir kız mıydı yani,23
9998,kocanın sana altın bir kolye aldığını sanıp,23


In [16]:
df_eightgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Eightgram_Selected_10000.xlsx")
df_eightgram_select

,eightgram,frequency
0,bir iki üç dört beş altı yedi sekiz,1060
1,dokuz sekiz yedi altı beş dört üç iki,513
2,sekiz yedi altı beş dört üç iki bir,497
3,iki üç dört beş altı yedi sekiz dokuz,411
4,üç dört beş altı yedi sekiz dokuz on,329
...,...,...
9995,başka bir yerde de patlama olacakmış anlıyor m...,18
9996,uygulama komitesi mi yoksa beni kullanacak ada...,18
9997,tuhaf bir şey değil mi insanın kendini suikastçı,18
9998,hayatındaki en büyük an dostum ve bir yerde,18


In [17]:
df_ninegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Ninegram_Selected_10000.xlsx")
df_ninegram_select

,ninegram,frequency
0,dokuz sekiz yedi altı beş dört üç iki bir,463
1,bir iki üç dört beş altı yedi sekiz dokuz,405
2,iki üç dört beş altı yedi sekiz dokuz on,287
3,on dokuz sekiz yedi altı beş dört üç iki,277
4,ve sen herkesin seni tanıdığı o yerde olmak is...,134
...,...,...
9995,sabah yedide buraya gel ve birlikte rakun avın...,13
9996,bunun ne anlama geldiği hakkında bir fikrin va...,13
9997,bence sen böyle bir kurumu idare edebilecek bi...,13
9998,farklı bir şey yapması için bir neden var mı,13


In [18]:
df_tengram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Tengram_Selected_10000.xlsx")
df_tengram_select

,tengram,frequency
0,umutsuzca birinin arkasından koşuyorum daha gü...,76
1,gece gündüz çifte park edilir insanlar naber k...,70
2,annem bana bazı şeyleri anlatmak için mutlaka ...,65
3,fakat dünyanın ona en çok ihtiyaç duyduğu anda...,64
4,zamanımızı geri almak istiyorum ama şans ve ka...,58
...,...,...
9991,mahkemeye gelip cinayet işlediği sırada beniml...,8
9992,bu olay yüzünden senin yüzünden pek çok insan ...,8
9993,batılı ve arap liderler arasındaki tarihi bir ...,8
9994,başımı belaya sokmam ve bir daha beni düşünmek...,8


#### Text Data Word-Word Group In Word Group

In [19]:
word_list = df_word_select["word"].to_list()
twogram_list = df_twogram_select["twogram"].to_list()
threegram_list = df_threegram_select["threegram"].to_list()
fourgram_list = df_fourgram_select["fourgram"].to_list()
fivegram_list = df_fivegram_select["fivegram"].to_list()
sixgram_list = df_sixgram_select["sixgram"].to_list()
sevengram_list = df_sevengram_select["sevengram"].to_list()
eightgram_list = df_eightgram_select["eightgram"].to_list()
ninegram_list = df_ninegram_select["ninegram"].to_list()
tengram_list = df_tengram_select["tengram"].to_list()

##### Word

In [20]:
df_word_in_two = word_in_wordgroup(word_list, df_twogram_select, "twogram", simple=True)
df_word_in_two

,word,twogram
0,bir,bir şey
1,bir,bu bir
2,bir,bir dakika
3,bir,bir şeyler
4,bir,başka bir
...,...,...
19911,delirdiniz,delirdiniz mi
19912,nyora,nyora duzenleme
19913,duzenleme,nyora duzenleme
19914,shoukanjuu,to shoukanjuu


In [21]:
df_word_in_two["word"].nunique()

3356

##### Twogram

In [22]:
df_two_in_three = word_in_wordgroup(twogram_list, df_threegram_select, "threegram", simple=True)
df_two_in_three.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_three

,twogram,threegram
0,bir şey,bir şey yok
1,bir şey,bir şey var
2,bir şey,başka bir şey
3,bir şey,bir şey değil
4,bir şey,bir şey mi
...,...,...
16488,olduğunu gördün,olduğunu gördün mü
16489,olduğunu gördün,ne olduğunu gördün
16490,bile bilmiyoruz,olduğunu bile bilmiyoruz
16491,iyi olmamıştım,daha iyi olmamıştım


In [23]:
df_two_in_four = word_in_wordgroup(twogram_list, df_fourgram_select, "fourgram", simple=True)
df_two_in_four.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_four

,twogram,fourgram
0,bir şey,bir şey var mı
1,bir şey,gereken bir şey var
2,bir şey,başka bir şey var
3,bir şey,bir şey daha var
4,bir şey,bir şey mi var
...,...,...
23396,olduğunu gördün,ne olduğunu gördün mü
23397,bile bilmiyoruz,ne olduğunu bile bilmiyoruz
23398,bile bilmiyoruz,olup olmadığını bile bilmiyoruz
23399,iyi olmamıştım,bu kadar iyi olmamıştım


In [24]:
df_two_in_five = word_in_wordgroup(twogram_list, df_fivegram_select, "fivegram", simple=True)
df_two_in_five.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_five

,twogram,fivegram
0,bir şey,başka bir şey var mı
1,bir şey,sana bir şey sorabilir miyim
2,bir şey,yapabileceğim bir şey var mı
3,bir şey,sana bir şey göstermek istiyorum
4,bir şey,söylemem gereken bir şey var
...,...,...
26831,sakın bunu,sakın bunu bir daha yapma
26832,sakın bunu,bir daha sakın bunu yapma
26833,iyi olmamıştım,hiç bu kadar iyi olmamıştım
26834,uzun değil,o kadar da uzun değil


In [25]:
df_two_in_six = word_in_wordgroup(twogram_list, df_sixgram_select, "sixgram", simple=True)
df_two_in_six.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_six

,twogram,sixgram
0,bir şey,sana söylemem gereken bir şey var
1,bir şey,söylemek istediğin bir şey var mı
2,bir şey,daha önce hiç böyle bir şey
3,bir şey,için yapabileceğim bir şey var mı
4,bir şey,sana söylemek istediğim bir şey var
...,...,...
20645,olduğunu gördün,ne kadar kolay olduğunu gördün mü
20646,olduğunu gördün,ne kadar güzel olduğunu gördün mü
20647,veya bir,ayakkabı boyası veya bir parça sakız
20648,veya bir,boyası veya bir parça sakız için


In [26]:
df_two_in_seven = word_in_wordgroup(twogram_list, df_sevengram_select, "sevengram", simple=True)
df_two_in_seven.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_seven

,twogram,sevengram
0,bir şey,bana söylemek istediğin bir şey var mı
1,bir şey,sizin için yapabileceğim bir şey var mı
2,bir şey,senin için yapabileceğim bir şey var mı
3,bir şey,bana söylemek istediğin bir şey mi var
4,bir şey,için yapabileceğim başka bir şey var mı
...,...,...
18126,veya bir,ayakkabı boyası veya bir parça sakız için
18127,veya bir,boyası veya bir parça sakız için bile
18128,veya bir,açık havada veya bir tarlada kalmalı ve
18129,bu gibi,bu gibi durumlarda genelde ne demek gerekir


In [27]:
df_two_in_eight = word_in_wordgroup(twogram_list, df_eightgram_select, "eightgram", simple=True)
df_two_in_eight.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_eight

,twogram,eightgram
0,bir şey,sizin için yapabileceğim başka bir şey var mı
1,bir şey,senin için yapabileceğim başka bir şey var mı
2,bir şey,sana söylemem gereken çok önemli bir şey var
3,bir şey,bana söylemek istediğin başka bir şey var mı
4,bir şey,bu isim sana bir şey ifade ediyor mu
...,...,...
19557,veya bir,ayakkabı boyası veya bir parça sakız için bile
19558,veya bir,veya bir çocuk gelip kitap hakkında sorular so...
19559,veya bir,derine temas eder veya bir nefes alırsan anında
19560,veya bir,et geri döner veya bir buluşma yeri vardır


In [28]:
df_two_in_nine = word_in_wordgroup(twogram_list, df_ninegram_select, "ninegram", simple=True)
df_two_in_nine.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_nine

,twogram,ninegram
0,bir şey,bu iyi bir şey mi kötü bir şey mi
1,bir şey,kılıcını ve onun kılıcını düşün başka bir şey ...
2,bir şey,daha önce hiç bu kadar güzel bir şey görmemiştim
3,bir şey,bir şey almaya dönen olması ihtimaline karşı g...
4,bir şey,bildiğimiz bir şey varsa o da bu arabayı isted...
...,...,...
22039,veya bir,bir şey olursa veya bir şey sorman gerekirse ara
22040,veya bir,peki konuşmaya veya bir şekilde iletişim kurma...
22041,veya bir,resepsiyon bölümünde bekleyebilirsiniz veya bi...
22042,bu gibi,ve bu gibi yerlerde böyle olay çıkarmak size hiç


In [29]:
df_two_in_ten = word_in_wordgroup(twogram_list, df_tengram_select, "tengram", simple=True)
df_two_in_ten.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_ten

,twogram,tengram
0,bir şey,bana sadece içinde şeker olmayan bir şey verin...
1,bir şey,niye konuşayım salaklar hayır bugün bana bir ş...
2,bir şey,efendi ani bu isim size bir şey ifade ediyor mu
3,bir şey,çünkü o zaman gerçekten bir şey olursa insanla...
4,bir şey,bir insan bir şeye bakar ve ondan bir şey çıka...
...,...,...
24878,baylar ve,baylar ve bayanlar bu bugüne kadar yaptığım en...
24879,etmek yok,rahatsız etmek yok hiçbir şeye dokunmak yok yo...
24880,veya bir,babam her kimse bir saatliğine veya bir aylığı...
24881,veya bir,aşk tek gecelik veya bir iki haftalık bir olay...


In [30]:
df_two_three_merge = pd.merge(df_twogram_select, df_two_in_three, how="left", on="twogram")
df_two_three_merge.drop_duplicates(inplace=True)
df_two_three_merge

,twogram,frequency,threegram
0,bir şey,859944,bir şey yok
1,bir şey,859944,bir şey var
2,bir şey,859944,başka bir şey
3,bir şey,859944,bir şey değil
4,bir şey,859944,bir şey mi
...,...,...,...
20787,çok acıktım,1317,NaN
20788,etmek yok,1317,NaN
20789,veya bir,1317,NaN
20790,evli misiniz,1316,NaN


In [31]:
df_two_three_merge["twogram"].nunique()

10000

In [32]:
df_two_four_merge = pd.merge(df_two_three_merge, df_two_in_four, how="left", on="twogram")
df_two_four_merge.drop_duplicates(inplace=True)
df_two_four_merge

,twogram,frequency,threegram,fourgram
0,bir şey,859944,bir şey yok,bir şey var mı
1,bir şey,859944,bir şey yok,gereken bir şey var
2,bir şey,859944,bir şey yok,başka bir şey var
3,bir şey,859944,bir şey yok,bir şey daha var
4,bir şey,859944,bir şey yok,bir şey mi var
...,...,...,...,...
897924,çok acıktım,1317,NaN,NaN
897925,etmek yok,1317,NaN,NaN
897926,veya bir,1317,NaN,NaN
897927,evli misiniz,1316,NaN,NaN


In [33]:
df_two_five_merge = pd.merge(df_two_four_merge, df_two_in_five, how="left", on="twogram")
df_two_five_merge.drop_duplicates(inplace=True)
df_two_five_merge

: 

: 

In [ ]:
dfs = [df_twogram_select, df_two_in_three, df_two_in_four, df_two_in_five, df_two_in_six, df_two_in_seven, df_two_in_eight, df_two_in_nine, df_two_in_ten]

In [ ]:
df_twogram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['twogram'], how='left'), dfs)

##### Threegram

In [ ]:
df_three_in_four = word_in_wordgroup(threegram_list, df_fourgram_select, "fourgram", simple=True)
df_three_in_four.rename(columns={"word":"threegram"}, inplace=True)
df_three_in_four

In [ ]:
df_four_in_five = word_in_wordgroup(fourgram_list, df_fivegram_select, "fivegram", simple=True)
df_four_in_five.rename(columns={"word":"fourgram"}, inplace=True)
df_four_in_five

In [ ]:
df_five_in_six = word_in_wordgroup(fivegram_list, df_sixgram_select, "sixgram", simple=True)
df_five_in_six.rename(columns={"word":"fivegram"}, inplace=True)
df_five_in_six

In [ ]:
df_six_in_seven = word_in_wordgroup(sixgram_list, df_sevengram_select, "sevengram", simple=True)
df_six_in_seven.rename(columns={"word":"sixgram"}, inplace=True)
df_six_in_seven

In [ ]:
df_seven_in_eight = word_in_wordgroup(sevengram_list, df_eightgram_select, "eightgram", simple=True)
df_seven_in_eight.rename(columns={"word":"sevengram"}, inplace=True)
df_seven_in_eight

In [ ]:
df_eight_in_nine = word_in_wordgroup(eightgram_list, df_ninegram_select, "ninegram", simple=True)
df_eight_in_nine.rename(columns={"word":"eightgram"}, inplace=True)
df_eight_in_nine

In [ ]:
df_nine_in_ten = word_in_wordgroup(ninegram_list, df_tengram_select, "tengram", simple=True)
df_nine_in_ten.rename(columns={"word":"ninegram"}, inplace=True)
df_nine_in_ten

##### Text Data Word-Word Group In Word Group Merge

In [ ]:
df_1_2_3_merge = pd.merge(df_word_in_two, df_two_in_three, how="left", on="twogram")
df_1_2_3_merge.drop_duplicates(inplace=True)
df_1_2_3_merge

In [ ]:
df_1_2_3_4_merge = pd.merge(df_1_2_3_merge, df_three_in_four, how="left", on="threegram")
df_1_2_3_4_merge.drop_duplicates(inplace=True)
df_1_2_3_4_merge

In [ ]:
df_1_2_3_4_5_merge = pd.merge(df_1_2_3_4_merge, df_four_in_five, how="left", on="fourgram")
df_1_2_3_4_5_merge.drop_duplicates(inplace=True)
df_1_2_3_4_5_merge

In [ ]:
df_1_2_3_4_5_6_merge = pd.merge(df_1_2_3_4_5_merge, df_five_in_six, how="left", on="fivegram")
df_1_2_3_4_5_6_merge.drop_duplicates(inplace=True)
df_1_2_3_4_5_6_merge

In [ ]:
df_1_2_3_4_5_6_7_merge = pd.merge(df_1_2_3_4_5_6_merge, df_six_in_seven, how="left", on="sixgram")
df_1_2_3_4_5_6_7_merge.drop_duplicates(inplace=True)
df_1_2_3_4_5_6_7_merge

In [ ]:
df_1_2_3_4_5_6_7_8_merge = pd.merge(df_1_2_3_4_5_6_7_merge, df_seven_in_eight, how="left", on="sevengram")
df_1_2_3_4_5_6_7_8_merge.drop_duplicates(inplace=True)
df_1_2_3_4_5_6_7_8_merge

In [ ]:
df_1_2_3_4_5_6_7_8_9_merge = pd.merge(df_1_2_3_4_5_6_7_8_merge, df_eight_in_nine, how="left", on="eightgram")
df_1_2_3_4_5_6_7_8_9_merge.drop_duplicates(inplace=True)
df_1_2_3_4_5_6_7_8_9_merge

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_merge = pd.merge(df_1_2_3_4_5_6_7_8_9_merge, df_nine_in_ten, how="left", on="ninegram")
df_1_2_3_4_5_6_7_8_9_10_merge.drop_duplicates(inplace=True)
df_1_2_3_4_5_6_7_8_9_10_merge

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_merge

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_merge["word"].nunique()

In [ ]:
word_count_result(df_1_2_3_4_5_6_7_8_9_10_merge,["twogram","threegram","fourgram","fivegram","sixgram","sevengram","eightgram","ninegram","tengram"])

In [ ]:
#df_1_2_3_4_5_6_7_8_9_10_merge.to_excel(f"{lang_folder.capitalize()}_Text_Selection_Data.xlsx", index=False)

In [ ]:
#df_1_2_3_4_5_6_7_8_9_10_freq_merge = pd.merge(df_1_2_3_4_5_6_7_8_9_10_merge, df_word_select, how="inner", on="word")
df_1_2_3_4_5_6_7_8_9_10_freq_merge = pd.merge(df_word_select, df_1_2_3_4_5_6_7_8_9_10_merge, how="left", on="word")
df_1_2_3_4_5_6_7_8_9_10_freq_merge.drop_duplicates(inplace=True)
#df_1_2_3_4_5_6_7_8_9_10_freq_merge.sort_values(by="frequency", inplace=True, ascending=False)
df_1_2_3_4_5_6_7_8_9_10_freq_merge

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_freq_merge["word"].nunique()

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_freq_merge["tengram"].nunique()

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_freq_merge["ninegram"].nunique()

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_freq_merge["eightgram"].nunique()

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"*_Selected_*.xlsx")
output_file

In [ ]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass

#### Temp

In [ ]:
df_text_file = pd.read_excel("Turkish_Text_Selection_Data.xlsx")
df_text_file

In [ ]:
df_text_file["word"].nunique()

In [ ]:
df_text_file.groupby("word")[["twogram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("twogram")[["threegram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("threegram")[["fourgram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("fourgram")[["fivegram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("fivegram")[["sixgram"]].agg(lambda x : set(x))

In [ ]:
df_test = df_text_file.groupby("fivegram")[["sixgram"]].agg(lambda x : set(x))
df_test.reset_index(inplace=True)
df_test

In [ ]:
fivegram_list = df_test["fivegram"].to_list()

In [ ]:
df_fivegram_select[~df_fivegram_select["fivegram"].isin(fivegram_list)]

In [ ]:
df_twogram_select[df_twogram_select["twogram"]=="söz vermedim"]

In [ ]:
df_threegram_select[df_threegram_select["threegram"]=="öyle bir söz"]

In [ ]:
df_text_file.groupby("sixgram")[["sevengram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("sevengram")[["eightgram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("eightgram")[["ninegram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("ninegram")[["tengram"]].agg(lambda x : set(x))

In [ ]:
df_ngram_concat = pd.concat([df_twogram_select, df_threegram_select, df_fourgram_select, df_fivegram_select], axis=1)
df_ngram_concat

In [ ]:
df_two_in_three = word_in_wordgroup_simple(df_ngram_concat, "twogram","threegram",threegram_sample)
df_two_in_four = word_in_wordgroup_simple(df_ngram_concat, "twogram","fourgram",fourgram_sample)
df_two_in_five = word_in_wordgroup_simple(df_ngram_concat, "twogram","fivegram",fivegram_sample)

In [ ]:
df_twogram_order_join_threegram = df_two_in_three.groupby(["twogram"])["threegram"].apply(", ".join).reset_index()
df_twogram_order_join_fourgram = df_two_in_four.groupby(["twogram"])["fourgram"].apply(", ".join).reset_index()
df_twogram_order_join_fivegram = df_two_in_five.groupby(["twogram"])["fivegram"].apply(", ".join).reset_index()  

In [ ]:
dfs = [df_twogram_order_join_threegram, df_twogram_order_join_fourgram, df_twogram_order_join_fivegram]

In [ ]:
df_ngram_sample_join_merge = reduce(lambda  left,right: pd.merge(left,right, on=['twogram'], how='outer'), dfs)
df_ngram_sample_join_merge.drop_duplicates(inplace=True)
df_ngram_sample_join_merge 

In [ ]:
df_ngram_sample_join_merge = pd.merge(df_ngram_sample_join_merge, df_twogram_select, how="right", on="twogram")
df_ngram_sample_join_merge.drop_duplicates(inplace=True)
df_ngram_sample_join_merge.rename(columns={"frequency":"two_freq"}, inplace=True)
df_ngram_sample_join_merge.sort_values(by="two_freq", ascending=False, inplace=True)
df_ngram_sample_join_merge.reset_index(drop=True, inplace=True)
df_ngram_sample_join_merge

In [ ]:
df_ngram_sample_join_merge.to_excel(f"{lang_folder.capitalize()}_{len(df_twogram_select)}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
Fourgram_{fivegram_sample}_Fivegram_Sample_With_{word_end}_Word_Join_Result.xlsx", index=False)

#### Adding Frequency For Ratio

In [ ]:
df_two_in_three_freq = pd.merge(df_two_in_three, df_threegram, how="left", on="threegram")
df_two_in_three_freq.drop_duplicates(inplace=True)
df_two_in_three_freq.rename(columns={"frequency":"three_freq"}, inplace=True)
df_two_in_three_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_three_freq

In [ ]:
df_two_in_four_freq = pd.merge(df_two_in_four, df_fourgram, how="left", on="fourgram")
df_two_in_four_freq.drop_duplicates(inplace=True)
df_two_in_four_freq.rename(columns={"frequency":"four_freq"}, inplace=True)
df_two_in_four_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_four_freq

In [ ]:
df_two_in_five_freq = pd.merge(df_two_in_five, df_fivegram, how="left", on="fivegram")
df_two_in_five_freq.drop_duplicates(inplace=True)
df_two_in_five_freq.rename(columns={"frequency":"five_freq"}, inplace=True)
df_two_in_five_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_five_freq

In [ ]:
df_ngram_sample_concat = pd.concat([df_twogram_select,df_two_in_three_freq, df_two_in_four_freq, df_two_in_five_freq], axis=1)
df_ngram_sample_concat

In [ ]:
df_ngram_sample_concat.to_excel(f"{lang_folder.capitalize()}_{len(df_twogram_select)}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
Fourgram_{fivegram_sample}_Fivegram_Sample_With_{word_end}_Word_Frequency_Result.xlsx", index=False)

#### Calculate Result And Select Ratio

In [ ]:
df_ngram_threegram_unique = df_ngram_sample_concat[["threegram","three_freq"]].drop_duplicates()
df_ngram_fourgram_unique = df_ngram_sample_concat[["fourgram","four_freq"]].drop_duplicates()
df_ngram_fivegram_unique = df_ngram_sample_concat[["fivegram","five_freq"]].drop_duplicates()

In [ ]:
twogram_result_freq = df_twogram_select["frequency"].sum()
threegram_result_freq = df_ngram_threegram_unique["three_freq"].sum()
fourgram_result_freq = df_ngram_fourgram_unique["four_freq"].sum()
fivegram_result_freq = df_ngram_fivegram_unique["five_freq"].sum()

In [ ]:
twogram_select_freq = df_twogram.iloc[twogram_select_start:twogram_select_end,]["frequency"].sum()
threegram_select_freq = df_threegram.iloc[threegram_select_start:threegram_select_end,]["frequency"].sum()
fourgram_select_freq = df_fourgram.iloc[fourgram_select_start:fourgram_select_end,]["frequency"].sum()
fivegram_select_freq = df_fivegram.iloc[fivegram_select_start:fivegram_select_end,]["frequency"].sum()

In [ ]:
(twogram_result_freq/twogram_select_freq)*100

In [ ]:
(threegram_result_freq/threegram_select_freq)*100

In [ ]:
(fourgram_result_freq/fourgram_select_freq)*100

In [ ]:
(fivegram_result_freq/fivegram_select_freq)*100

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{len(df_twogram_select)}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
Fourgram_{fivegram_sample}_Fivegram_Sample_*_Result.xlsx")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Temp

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

# word all usage in twogram
word_use_num_min = 1  # word usage in selected twograms 
word_use_num_max = 4

In [ ]:
def word_usage_result(word_list, df_target, target_column, word_usage_min, word_usage_max): # word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max)
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        #opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value])
            #result_list_select.append([target_value,opt_value])  
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}"])
    #df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    #df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [ ]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [ ]:
def word_in_wordgroup_simple(df, source_column, target_column, word_sample_num):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [ ]:
word_list = df_word_select["word"].to_list()

In [ ]:
df_two_three = pd.read_excel("/home/kurubal/Downloads/Twogram Threegram.xlsx")
df_two_three

In [ ]:
word_count_result(df_two_three,["threegram"])

In [ ]:
df_two_three["twogram"].count()

In [ ]:
df_two_three["threegram"].count()

In [ ]:
df_two_in_three = word_in_wordgroup_simple(df_two_three, "twogram","threegram",threegram_sample)
df_two_in_three

In [ ]:
set_all_twogram = set(df_two_three["twogram"].dropna().to_list())
set_twogram_in_threegram = set(df_two_in_three["twogram"].to_list())

In [ ]:
set_all_twogram.difference(set_twogram_in_threegram)

In [ ]:
set_threegram_cover_twogram = set(df_two_in_three["threegram"].to_list())
df_selected_threegram = pd.DataFrame(set_threegram_cover_twogram,columns=["threegram"])
df_selected_threegram

In [ ]:
df_selected_threegram.to_excel("Threegram_Selected.xlsx", index=False)

In [ ]:
word_count_result(df_selected_threegram,["threegram"])

In [ ]:
df_threegram_word_usage_select = word_usage_result(word_list, df_selected_threegram, "threegram", word_use_num_min, word_use_num_max)
df_threegram_word_usage_select

In [ ]:
word_count_result(df_threegram_word_usage_select,["threegram"])